In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 28800/28800 [00:01<00:00, 23939.50it/s]


In [3]:
DF.cpu.unique()

array(['Intel(R) Xeon(R) CPU E5-2695 v3 @ 2.30GHz',
       'AMD EPYC 7453 28-Core Processor',
       'Intel(R) Xeon(R) CPU E5-2695 v4 @ 2.10GHz',
       'Intel(R) Xeon(R) CPU E5-2699 v3 @ 2.30GHz',
       'Intel(R) Xeon(R) Platinum 8352V CPU @ 2.10GHz',
       'AMD EPYC 9654 96-Core Processor', '', None], dtype=object)

In [16]:
COLUMNS = [
    # "data_pruning",
    # "facts",
    "optimiser",
    # "features",
    "iterations",
    "feature_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "equivalent-weighted"]
    # df = df[df.feature_pruning.isin({"none", "collapse-all"})]
    df = df[df.feature_pruning != "collapse-all"]
    return df

dfs = {}
for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    # df = df.fillna(0)
    # replace 0 with NaN
    # df = df.replace(0, -1)

    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)
    dfs[metric] = df

solved


domain                                  blocksworld  childsnack  ferry  \
optimiser iterations feature_pruning                                     
rank-svm  2          0none                       73          25     71   
                     1collapse-all-x             70          29     71   
                     2collapse-layer             67          28     70   
                     3collapse-layer-x           47          31     69   
          4          0none                       69          32     68   
                     1collapse-all-x             65          37     69   
                     2collapse-layer             65          38     70   
                     3collapse-layer-x           47          31     68   
svr       2          0none                       32          18     68   
                     1collapse-all-x             31          18     71   
                     2collapse-layer             33          18     69   
                     3collapse-layer-x           48          22     67   
          4          0none                       61          31     68   
                     1collapse-all-x             56          18     69   
                     2collapse-layer             32          20     70   
                     3collapse-layer-x           46          22     68   

domain                                  floortile  miconic  rovers  satellite  \
optimiser iterations feature_pruning                                            
rank-svm  2          0none                      2       85      39         32   
                     1collapse-all-x            2       90      41         31   
                     2collapse-layer            3       90      40         33   
                     3collapse-layer-x          3       90      41         39   
          4          0none                      2       81       0          0   
                     1collapse-all-x            2       90      42          0   
                     2collapse-layer            2       90      39          0   
                     3collapse-layer-x          3       90      41         40   
svr       2          0none                      2       87      41         41   
                     1collapse-all-x            2       90      45         41   
                     2collapse-layer            2       90      46         42   
                     3collapse-layer-x          2       90      43         51   
          4          0none                      2       82      35         48   
                     1collapse-all-x            2       90      39         45   
                     2collapse-layer            2       90      40         47   
                     3collapse-layer-x          2       90      44         53   

domain                                  sokoban  spanner  transport  total  
optimiser iterations feature_pruning                                        
rank-svm  2          0none                   36       68         31    462  
                     1collapse-all-x         35       64         30    463  
                     2collapse-layer         34       68         31    464  
                     3collapse-layer-x       35       66         45    466  
          4          0none                   34       66         32    384  
                     1collapse-all-x         35       57         32    429  
                     2collapse-layer         34       67         33    438  
                     3collapse-layer-x       35       68         45    468  
svr       2          0none                   35       67         30    421  
                     1collapse-all-x         37       67         27    429  
                     2collapse-layer         35       66         31    432  
                     3collapse-layer-x       36       67         30    456  
          4          0none                   34       66         30    457  
                     1collapse-all-x         37      

In [14]:
df = dfs["solved"]
latex = df.to_latex()
latex = latex.replace("_", " ")
for domain in DOMAINS:
    latex = latex.replace(domain, '\\' + domain[:2] + "{}")
latex = latex.replace("total", r"$\Sigma$")
latex = latex.replace("domain", "")
for k, v in FEATURE_GENERATION_PREFIX.items():
    to_replace = f"{v}{k}"
    latex = latex.replace(to_replace, f"\\{k}{{}}".replace("-", ""))
latex = latex.replace(r"\begin{tabular}", r"\begin{tabularx}{\textwidth}")
latex = latex.replace(r"rrrrrrrrrrr}", r"Y Y Y Y Y Y Y Y Y Y Y}")
latex = latex.replace(r"\end{tabular}", r"\end{tabularx}")
latex = latex.replace(r"\cline{1-14} \cline{2-14}", r"\midrule")
latex = latex.replace(r"\midrule" + "\n" + r"\bottomrule", r"\bottomrule")
latex = latex.replace(r"\cline", r"\cmidrule")
with open("/home/dzc/manuscripts/socs-25-redundant/tables/coverage.tex", "w") as f:
    f.write(latex)

In [6]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        # print(len(df))
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"] + " - " + df["data_pruning"] # + " - " + df["feature_pruning"]
        # df["config"] = " _ ".join([df[f] for f in ["features", "iterations", "optimiser", "data_pruning", "feature_pruning"]])
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="feature_pruning",
            line_dash="feature_pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        fig.update_xaxes(categoryorder='array', categoryarray=PROBLEMS)
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)